In [1]:
#@title 최종 AI 추천 서버 실행 (지능형 폴백 + 상세 정보 입력 통합)

# --- 1. 필수 라이브러리 설치 ---
# T4 GPU 환경에 맞게 bitsandbytes를 포함하여 설치합니다.
!pip install -q -U transformers peft accelerate trl datasets huggingface_hub fastapi "uvicorn[standard]" pyngrok bitsandbytes "pandas==2.2.2"


In [2]:
# --- 2. Google Drive 마운트 및 경로 설정 ---
from google.colab import drive
import os
print("📂 Google Drive를 마운트합니다...")
drive.mount('/content/drive')
PROJECT_DIR = "/content/drive/MyDrive/Colab Notebooks/fashion_project"
os.chdir(PROJECT_DIR)
print(f"✅ 작업 디렉토리를 '{PROJECT_DIR}'(으)로 변경했습니다.")


📂 Google Drive를 마운트합니다...
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ 작업 디렉토리를 '/content/drive/MyDrive/Colab Notebooks/fashion_project'(으)로 변경했습니다.


In [3]:
# --- 3. Hugging Face 로그인 ---
from huggingface_hub import login
print("\n--- 🔐 Hugging Face 로그인이 필요합니다 ---")
login()
print("✅ 로그인 성공!")



--- 🔐 Hugging Face 로그인이 필요합니다 ---


✅ 로그인 성공!


In [4]:
# --- 4. 모델 폴더 압축 해제 ---
ZIP_FILE = "gemma-fashion-dpo-final-v3.zip"
ADAPTER_FOLDER = "gemma-fashion-dpo-final-v3"
if not os.path.exists(ADAPTER_FOLDER):
    print(f"\n'{ZIP_FILE}' 파일의 압축을 해제합니다...")
    !unzip -q -o "{ZIP_FILE}" -d .
else:
    print(f"\n'{ADAPTER_FOLDER}' 폴더가 이미 존재합니다.")
ADAPTER_PATH = os.path.join(PROJECT_DIR, ADAPTER_FOLDER)
print(f"✅ 모델 경로 확인: {ADAPTER_PATH}")


'gemma-fashion-dpo-final-v3' 폴더가 이미 존재합니다.
✅ 모델 경로 확인: /content/drive/MyDrive/Colab Notebooks/fashion_project/gemma-fashion-dpo-final-v3


In [5]:
# --- 5. 최종 serve_final.py 파일 자동 생성 ---
%%writefile serve_final.py

import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import PeftModel
import pandas as pd
from itertools import product
import time
import random
from fastapi import FastAPI
from pydantic import BaseModel, Field
from typing import List, Dict, Any
import uvicorn
import logging

# --- 기본 설정 ---
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)
app = FastAPI()

# --- 전역 변수 ---
model, tokenizer, styles_info, positive_token_id, negative_token_id = None, None, None, None, None
device = "cuda" if torch.cuda.is_available() else "cpu"

# --- 서버 시작 시 모델 로딩 ---
@app.on_event("startup")
def load_resources():
    global model, tokenizer, styles_info, positive_token_id, negative_token_id
    logger.info("✅ 최종 DPO 모델 로딩을 시작합니다...")
    # Colab 노트북이 자동으로 찾아준 절대 경로를 사용합니다.
    adapter_path = "ADAPTER_PATH"
    csv_path = "CSV_PATH"

    logger.info(f"Using adapter path: {{adapter_path}}")
    quantization_config = BitsAndBytesConfig(load_in_4bit=True, bnb_4bit_quant_type="nf4", bnb_4bit_compute_dtype=torch.bfloat16)
    base_model = AutoModelForCausalLM.from_pretrained("google/gemma-2b-it", quantization_config=quantization_config, device_map="auto")
    model = PeftModel.from_pretrained(base_model, adapter_path)
    model.eval()
    tokenizer = AutoTokenizer.from_pretrained("google/gemma-2b-it")
    tokenizer.pad_token = tokenizer.eos_token
    positive_token_id = tokenizer.encode("긍정적", add_special_tokens=False)[0]
    negative_token_id = tokenizer.encode("부정적", add_special_tokens=False)[0]
    logger.info("✅ 모델 및 토크나이저 로딩 완료!")
    try:
        styles_df = pd.read_csv("styles.csv", on_bad_lines='skip', dtype={'id': int}).set_index('id')
        styles_info = styles_df.to_dict('index')
        logger.info("✅ styles.csv 로드 완료!")
    except FileNotFoundError:
        logger.warning("⚠️ styles.csv 파일을 찾을 수 없어, 빈 데이터로 시작합니다.")
        styles_info = {}

# --- 헬퍼 함수 및 데이터 모델 ---
def get_item_description(item_id, temp_styles_info):
    info = temp_styles_info.get(item_id, {})
    season = info.get('season', 'All')
    return f"{info.get('baseColor', '')} {info.get('articleType', '')} ({season})".strip()

class ClosetItem(BaseModel):
    id: int; gender: str; masterCategory: str; subCategory: str
    articleType: str; baseColor: str; season: str; usage: str

class OutfitRequest(BaseModel):
    closet: List[ClosetItem]
    event: str; temperature: float; condition: str; gender: str

@app.post("/recommend_outfit")
def recommend_outfit(req: OutfitRequest):
    start_time = time.time()

    # 1. 입력 데이터 처리
    request_closet_info = {item.id: item.dict() for item in req.closet}
    temp_styles_info = styles_info.copy()
    temp_styles_info.update(request_closet_info)
    closet_ids = [item.id for item in req.closet]

    # 2. 컨텍스트 기반 아이템 필터링
    temp = req.temperature
    if temp <= 15: suitable_seasons = ['Winter', 'Fall', 'All Season']
    else: suitable_seasons = ['Summer', 'Spring', 'All Season']
    target_usage = ['Casual', 'Smart Casual']
    if req.event in ["Business Meeting", "Formal Dinner"]: target_usage = ['Formal', 'Smart Casual']
    elif req.event in ["Gym Workout", "Hiking", "Sports"]: target_usage = ['Sports', 'Active']

    filtered_closet_ids = [
        item_id for item_id in closet_ids
        if item_id in temp_styles_info and
        (temp_styles_info[item_id].get('season') in suitable_seasons) and
        (temp_styles_info[item_id].get('usage') in target_usage)
    ]
    if not filtered_closet_ids:
        return {"error": "현재 상황에 맞는 아이템이 옷장에 없습니다."}

    # 3. 개별 아이템 AI 평가
    item_prompts = [f"상황: {req.gender}, {req.temperature}°C, {req.condition}, {req.event}\n옷: {get_item_description(item_id, temp_styles_info)}\n결과:" for item_id in filtered_closet_ids]
    inputs = tokenizer(item_prompts, return_tensors="pt", padding=True).to(device)
    with torch.no_grad(): outputs = model(**inputs)
    sequence_lengths = inputs['attention_mask'].sum(dim=1) - 1
    last_token_logits = outputs.logits[torch.arange(len(filtered_closet_ids)), sequence_lengths]
    logits_for_scoring = last_token_logits[:, [negative_token_id, positive_token_id]]
    probs = torch.softmax(logits_for_scoring, dim=-1)
    item_scores = probs[:, 1].tolist()
    scored_items = {filtered_closet_ids[i]: item_scores[i] for i in range(len(filtered_closet_ids))}

    # 4. 부위별 분류 및 Top-K 선정
    classified_items = { 'Topwear': [], 'Bottomwear': [], 'Outerwear': [], 'Footwear': [], 'Full Body': [] }
    outerwear_types = ['Jackets', 'Blazers', 'Waistcoat', 'Coat', 'Shrug', 'Cardigan', 'Nehru Jackets', 'Rain Jacket', 'Sweaters']
    full_body_types = ['Dresses', 'Jumpsuit']
    for item_id in filtered_closet_ids:
        info = temp_styles_info[item_id]; part = None
        if info.get('articleType') in full_body_types: part = 'Full Body'
        elif info.get('articleType') in outerwear_types: part = 'Outerwear'
        elif info.get('subCategory') == 'Topwear': part = 'Topwear'
        elif info.get('subCategory') == 'Bottomwear': part = 'Bottomwear'
        elif info.get('masterCategory') == 'Footwear': part = 'Footwear'
        if part: classified_items[part].append(item_id)

    top_items_by_part = { part: sorted(items, key=lambda x: scored_items.get(x, 0), reverse=True)[:5] for part, items in classified_items.items() }

    # --- ★ NEW: 지능형 폴백(Fallback) 로직 ---
    def generate_combinations(items_pool):
        base = list(product(items_pool.get('Topwear', []), items_pool.get('Bottomwear', []), items_pool.get('Footwear', [])))
        outer = list(product(items_pool.get('Topwear', []), items_pool.get('Bottomwear', []), items_pool.get('Outerwear', []), items_pool.get('Footwear', [])))
        full_body = list(product(items_pool.get('Full Body', []), items_pool.get('Footwear', [])))
        return [c for c in base + outer + full_body if c]

    # 1차 시도: 최상위 아이템으로 조합 생성
    all_combinations = generate_combinations(top_items_by_part)

    # 2차 시도: 1차 실패 시, 필터링 통과한 모든 아이템으로 재시도
    if not all_combinations:
        logger.warning("상위 아이템으로 조합 생성 실패. 필터링된 전체 아이템으로 재시도합니다.")
        all_combinations = generate_combinations(classified_items)

    if not all_combinations:
        return {"error": "옷장의 아이템들로 유효한 조합을 만들 수 없습니다."}

    # 5. 최종 조합 평가
    prompts = [ f"상황: {req.gender}, {req.temperature}°C, {req.condition}, {req.event}\n옷: {', '.join([get_item_description(id, temp_styles_info) for id in combo])}\n결과:" for combo in all_combinations ]
    inputs = tokenizer(prompts, return_tensors="pt", padding=True).to(device)
    with torch.no_grad(): outputs = model(**inputs)
    sequence_lengths = inputs['attention_mask'].sum(dim=1) - 1
    last_token_logits = outputs.logits[torch.arange(len(all_combinations)), sequence_lengths]
    logits_for_scoring = last_token_logits[:, [negative_token_id, positive_token_id]]
    probs = torch.softmax(logits_for_scoring, dim=-1)
    all_scores = probs[:, 1].tolist()
    best_score_index = all_scores.index(max(all_scores))
    best_score = all_scores[best_score_index]
    best_combo = all_combinations[best_score_index]
    best_combo_ids = [int(id_val) for id_val in best_combo if id_val is not None]
    outfit_str = ", ".join([get_item_description(id, temp_styles_info) for id in best_combo_ids])
    best_outfit_info = {"description": outfit_str, "ids": best_combo_ids}

    explanation = f"긍정적: {req.temperature}°C의 {req.condition} 날씨에 진행되는 {req.event}에 가장 잘 어울리는 스타일입니다."
    end_time = time.time()
    return {"best_combination": best_outfit_info, "explanation": explanation, "best_score": best_score, "processing_time": end_time - start_time}


Overwriting serve_final.py


In [6]:
# --- 6. 서버 실행 및 ngrok 터널 생성 ---
import threading, time, requests, uvicorn
from pyngrok import ngrok

def run_app():
  uvicorn.run("serve_final:app", host="0.0.0.0", port=8000, log_level="info")

thread = threading.Thread(target=run_app)
thread.start()
print("\n✅ FastAPI 서버가 백그라운드에서 실행을 시작했습니다...")

print("⏳ 서버가 모델을 로딩하고 준비될 때까지 대기 중입니다...")
start_time = time.time()
server_ready = False
while time.time() - start_time < 600:
    try:
        response = requests.get("http://localhost:8000/", timeout=5)
        if response.status_code == 404:
            print("✅ 서버가 응답하기 시작했습니다!")
            server_ready = True
            break
    except requests.exceptions.ConnectionError: time.sleep(5); print("...")
if not server_ready:
    print("❌ 10분이 지나도 서버가 준비되지 않았습니다. 런타임을 재시작하고 다시 시도해주세요.")
else:
    NGROK_TOKEN = "31HHkLEWGt90qDRoAWd6BqiGL9K_5fJsXN7LgijbtcAVwkwAS" # 🚨
    ngrok.set_auth_token(NGROK_TOKEN)
    ngrok.kill()
    public_url = ngrok.connect(8000)
    print(f"\n🎉 서버가 성공적으로 생성되었습니다. API 주소: {public_url}")
    print("이제 이 주소를 test_api.py에 넣고 로컬에서 테스트를 실행하세요.")

thread.join()


✅ FastAPI 서버가 백그라운드에서 실행을 시작했습니다...
⏳ 서버가 모델을 로딩하고 준비될 때까지 대기 중입니다...
...
...
...
...
...


INFO:     Started server process [2678]
INFO:     Waiting for application startup.


...


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

...
...
...
...
...


KeyboardInterrupt: 

INFO:     Started server process [17286]
INFO:     Waiting for application startup.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

ERROR:    Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_http.py", line 409, in hf_raise_for_status
    response.raise_for_status()
  File "/usr/local/lib/python3.12/dist-packages/requests/models.py", line 1026, in raise_for_status
    raise HTTPError(http_error_msg, response=self)
requests.exceptions.HTTPError: 404 Client Error: Not Found for url: https://huggingface.co/gemma-fashion-dpo-final-v3/resolve/main/adapter_config.json

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/peft/config.py", line 262, in _get_peft_type
    config_file = hf_hub_download(
                  ^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_validators.py", line 114, in _inner_fn
    return fn(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/huggingface_hub/fi

🎉 서버 생성 완료. API 주소: NgrokTunnel: "https://8f00a8e614fc.ngrok-free.app" -> "http://localhost:8000"



✅ FastAPI 서버가 백그라운드에서 실행을 시작했습니다...
⏳ 서버가 모델을 로딩하고 준비될 때까지 대기 중입니다...
...


INFO:     Started server process [17286]
INFO:     Waiting for application startup.


...


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

...


ERROR:    Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_http.py", line 409, in hf_raise_for_status
    response.raise_for_status()
  File "/usr/local/lib/python3.12/dist-packages/requests/models.py", line 1026, in raise_for_status
    raise HTTPError(http_error_msg, response=self)
requests.exceptions.HTTPError: 404 Client Error: Not Found for url: https://huggingface.co/gemma-fashion-dpo-final-v3/resolve/main/adapter_config.json

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/peft/config.py", line 262, in _get_peft_type
    config_file = hf_hub_download(
                  ^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_validators.py", line 114, in _inner_fn
    return fn(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/huggingface_hub/fi

...
...
...
...
...
...
...
...
...
...
...


KeyboardInterrupt: 

SyntaxError: incomplete input (ipython-input-1976066550.py, line 22)

INFO:     Started server process [1626]
INFO:     Waiting for application startup.


🎉 서버 생성 완료. API 주소: NgrokTunnel: "https://4b48e71ab8ed.ngrok-free.app" -> "http://localhost:8000"
